In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.optim import SGD, Adam
from torchvision import datasets
!pip install torchsummary
from torchsummary import summary

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# input data for cnn
X_train = torch.tensor([
# sample 1
 [[[1,2,3,4],[2,3,4,5],
 [5,6,7,8],[1,3,4,5]]],
# sample 2
 [[[-1,2,3,-4],[2,-3,4,5],
 [-5,6,-7,8],[-1,-3,-4,-5]]]
 ]).to(device).float()

X_train /= 8 # scalling
print(X_train.shape)
# targets
y_train = torch.tensor([0,1]).to(device).float()

torch.Size([2, 1, 4, 4])


In [4]:
# model architecture
def get_model():
    model = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3),
        nn.MaxPool2d(kernel_size=2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(in_features=1, out_features=1),
        nn.Sigmoid(),
    ).to(device)
    loss_func = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=0.001)
    return model, loss_func, optimizer

In [5]:
# model summary 
model, loss_func, optimizer = get_model()
summary(model, X_train.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 1, 2, 2]              10
         MaxPool2d-2              [-1, 1, 1, 1]               0
              ReLU-3              [-1, 1, 1, 1]               0
           Flatten-4                    [-1, 1]               0
            Linear-5                    [-1, 1]               2
           Sigmoid-6                    [-1, 1]               0
Total params: 12
Trainable params: 12
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [6]:
# train batches
def train_batch(x, y, model, optimizer, loss_func):
    model.train()
    pred = model(x)
    batch_loss = loss_func(pred.squeeze(0), y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [7]:
# load data
train_dl = DataLoader(TensorDataset(X_train, y_train))

In [8]:
# model training
for epoch in range(3000):
    for ix, batch in enumerate(iter(train_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_func)

In [9]:
model(X_train[:1])

tensor([[0.5063]], device='cuda:0', grad_fn=<SigmoidBackward0>)